<font size="5"><b>Lecture 09: Machine Learning 3/3 (exercises)</b></font>

# EX: Semantic classification of satellite image (part 2/2)

<div class="alert alert-warning">
<b>This exercice has been developped by <u>Andreas Ley</u> and <u>Ronny Hänsch</u> (TU-Berlin) in the framework of the GEO.X 2017 Autumn school.<br>
<br>
The exercice is designed to implement the classification of a satellite image (Sentinel-2) into semantic labels defining land use: forest, fields, urban, water.<br>
<br>
The exercice is split into 2 parts: this first part (lecture 08) will implement a PCA analisis on the image crops in order to reduce dimensionality. The second part (lecture 09) will use the outputs of the PCA and implement a SVM to classify the pixels into different land use classes.
    </b>
</div>

<div class="alert alert-warning">
<b><u>Description of the dataset</u></b>:<br>
<br>
The data stored in the file "images/s2_training_data.npz" contains image crops of a Sentinel-2 multispectral image, and some semantic labels of land use.<br>
    <br>
    The file can be loaded using <a href="https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load" target="_blank">numpy.load()</a>, and contains 2 parts:<br>
    <ul>
        <li><u>data</u>: contains 20,000 patches (crops), each of 15x15 pixels and 4 channels (R + G + B + shortwave-infrared). The data is organized as one numpy array of shape (20000, 4, 15, 15), where the first dimension are the instances, the next dimension are the channels, and the latter two are width and height.<br> The first 5000 crops contain forest in the central pixel, the next 5000 contain fields/lower vegetation, the next 5000 are urban areas, and the last 5000 are water.</li>
        <li><u>labels</u>: contains semantic labels of land use (forest, fields, urban, water), which will be later used for training the classifier.</li>
    </ul>   
</div>

<div class="alert alert-info">
In this second part of the exercice, in addition to the training data you will be using two other files:<br>
    <ul>
        <li><u>s2_testing_data.npz</u>: the file is structured like the training file "s2_training_data.npz", and will be used to evaluate our SVM classifier.</li>
        <br>
        <li><u>s2_application_data.npz</u>: the file only contains a "data" field, which stores a single Sentinel-2 image of 512x512 pixels, with 4 channels (R+G+B+SWIR). This image will be used to apply the trained SVM classifier in order to classify the pixels into the different land use classes (forest, fields, urban, water).</li>
    </ul>   
</div>

## define functions developped in part 1 (PCA)

<div class="alert alert-success">
    The datasets contain 5k crops per class. Each crop is 15x15 pixels in size with 4 channels per pixel. The label of the central pixel is given in a separate array. Since the input dimensionality of each crop is quite high (4x15x15 = 900), we want to compress it using PCA.<br>
    The following function have been developped in the 1st part of the exercice.
</div>

In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [45]:
def show_raw_image(img):

    img2 = np.log(img[[2,1,0],:,:])
    
    img2[0,:,:] = img2[0,:,:].copy() * 1.05303 + -6.32792
    img2[1,:,:] = img2[1,:,:].copy() * 1.74001 + -10.8407
    img2[2,:,:] = img2[2,:,:].copy() * 1.20697 + -6.73016

    img2 = np.clip(img2 / 6 + 0.5, 0.0, 1.0)

    plt.imshow(np.transpose(img2, (1, 2, 0)))
    #plt.show()


def compute_mean_PCs(X):
    mean = np.mean(X, 0) # Second parameter is the axis along which we want to average (0 == across instances)
    mean_free = X-mean

    vars_per_img = X.shape[1] * X.shape[2] * X.shape[3]
    num_imgs = mean_free.shape[0]

    # Flatten data in to vectors
    mean_free_vectorized = np.reshape(mean_free, (num_imgs, vars_per_img))

    # Increase this to speed up debugging
    covar_subsampling = 2

    # Accumulate covar matrix
    covar = np.zeros((vars_per_img, vars_per_img))
    print("Image: 0")
    for i in range(0, num_imgs, covar_subsampling):
        print("\rImage: {}".format(i))
        covar += np.outer(mean_free_vectorized[i,:], mean_free_vectorized[i,:])

    covar /= num_imgs/covar_subsampling

    eig_val, eig_vec = np.linalg.eig(covar)

    # Sort by importance
    idx = np.argsort(eig_val)[::-1]
    eig_vec = eig_vec[:,idx]
    eig_val = eig_val[idx]

    # Reshape data back into images. Note that eig_vec is the transpose of what you might expect it to be.
    principal_components = np.transpose(eig_vec, (1,0)).reshape((vars_per_img, X.shape[1], X.shape[2], X.shape[3]))

    return mean, principal_components, eig_val



def show_first_principal_components(pcs):

    f, axarr = plt.subplots(8,8)
    for i in range(0,8):
        for j in range(0,8):
            img2 = pcs[i*8+j,[2,1,0],:,:]
            img2 = np.clip(img2 * 10 + 0.5, 0.0, 1.0)
            axarr[i,j].imshow(np.transpose(img2, (1, 2, 0)))

    plt.show()

def compute_features(X, mean, principal_components, count):
    X_mean_free = X - mean
    features = np.zeros((X.shape[0], count))
    for i in range(0, X.shape[0]):
        for j in range(0, count):
            # Note: The [i,:,:,:] is being very explicit here. [i] would also work.
            features[i,j] = X_mean_free[i,:,:,:].flatten().dot(principal_components[j,:,:,:].flatten())
    return features
    
def reconstruct_image(feature, mean, principal_components):
    reconstruction = np.copy(mean)
    for i in range(0, feature.shape[0]):
        reconstruction += feature[i] * principal_components[i,:,:,:]
    return reconstruction    

In [7]:
def testAccuracy(classifier, test_X, test_Y):
    inferred_Y = classifier.predict(test_X)
    return np.mean(test_Y == inferred_Y)

## run PCA on training_data

<div class="alert alert-success">
    Use the functions implemented for the PCA exercise to compute the mean and principal components on the training data (s2_training_data.npz).<br>
    Compute a 16-dimensional feature vector for each training sample by using the "compute_features" function.
</div>

## train a SVM on PCA features

<div class="alert alert-success">
    Train a support vector machine on the computed features and the corresponding labels. Use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html" target="_blank">sklearn.svm.SVC()</a>.
</div>

## compute the training accuracy of the classifier

<div class="alert alert-success">
    Compute the accuracy on the training dataset using the function testAccuracy() defined above.
</div>

## load test dataset and evaluate classifier

<div class="alert alert-success">
    1. Load the test dataset from the file "s2_testing_data.npz".<br>
    2. Compute features for it as well (remember to compute the logarithm of the data first) and evaluate the classifier on the test dataset by testing it's accuracy.<br>
    3. Optional: try out the training accuracy for 16, 32, and 256 principal components. What is the test accuracy for those? How long does training/inference take (roughly)?
</div>

## classify pixels in application dataset

<div class="alert alert-success">
    1. Load the image "data/s2_application_data.npz" and plot using show_raw_image().<br>
    Have a look at it: can you spot the forest, urban, field, and water regions?<br>
    <br>
    2. Compute features on the training dataset with 64 components, and train SVM classifier with these features.<br>
    <br>
    3. Classify the image pixels into the four semantic classes (Forest, Field, Urban, Water):<br>
    <br>
    Use a "sliding window" approach where you crop a window of 15x15 pixels from the source image, that you will use to infer the label of the central pixel. To do so, compute the crop features, and use these to "predict" the pixel class with your SVM classifier. Store the predicted pixel value in an array "application_labels" (shape = (application_X.shape[1]-15, application_X.shape[2]-15)). Then shift the crop by one pixel and repeat these steps in order to classify the next pixel. Continue sliding the crop across the entire image in order to classify the entire image.<br>
<u>Note</u>: compute_features expects an array of crops instead of a single crop which means that you need to turn your crop into an array containing just one crop.<br>
    <br>
    4. Render the final label map with: plt.imshow(application_labels).<br>
    Is the classifier able to correctly label the different regions (forest, urban, field, water) you spotted in the image?<br>
    How could you improve it?
</div>